In [536]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
import statsmodels.api as sm
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats 
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor

In [538]:
df = pd.read_csv("resume.csv") 

In [539]:
df = df.dropna(subset=['russian_salary'])

In [541]:
df.columns

Index(['id_resume', 'title', 'created_at', 'updated_at', 'age', 'gender',
       'salary', 'currency', 'photo', 'total_experience', 'citizenship',
       'area', 'level_education', 'university', 'count_additional_courses',
       'employments', 'experience', 'language_eng', 'language_zho',
       'schedules', 'skill_set', 'is_driver', 'professional_roles', 'url',
       'russian_salary'],
      dtype='object')

In [543]:
df = df.drop(columns=[
    'id_resume', 'created_at', 'updated_at', 'currency', 'salary', 
    'url'
])

In [545]:
df.head()

,title,age,gender,photo,total_experience,citizenship,area,level_education,university,count_additional_courses,employments,experience,language_eng,language_zho,schedules,skill_set,is_driver,professional_roles,russian_salary
10,Аналитик,NaN,female,False,243.0,{Россия},Санкт-Петербург,Высшее,"{""Московский коммерческий университет"",Москва}",NaN,"{""Полная занятость""}","{Минимакс,Минимакс}",NaN,NaN,"{""Полный день""}","{""Аналитика данных""}",True,{Аналитик},100000.0
20,Бизнес-аналитик,NaN,male,False,NaN,{Россия},Натухаевская,Среднее специальное,{},NaN,"{""Полная занятость""}",{},NaN,NaN,"{""Полный день"",""Удаленная работа""}","{Ответственность,""MS PowerPoint"",""Подготовка п...",False,{Бизнес-аналитик},80000.0
132,Аналитик,26.0,male,True,31.0,{Россия},Москва,Неоконченное высшее,"{""Московский государственный университет имени...",NaN,"{""Полная занятость"",""Частичная занятость"",""Про...",{Техноласточка},c1,NaN,"{""Полный день"",""Сменный график"",""Гибкий график...","{""Аналитическое мышление"",Python,""Английский я...",False,{Аналитик},160000.0
144,Системный аналитик,27.0,male,False,88.0,{Россия},Москва,Магистр,"{""Российская академия народного хозяйства и го...",NaN,"{""Полная занятость""}","{Ингосстрах,""ООО «дистанционная медицина»""}",NaN,NaN,"{""Полный день""}","{""Ruby On Rails"",Ruby,Json,UML,Redis,Kafka,Nod...",False,"{Аналитик,""Системный аналитик""}",250000.0
164,Аналитик,51.0,male,False,355.0,{Россия},Москва,Высшее,"{""Серпуховской военный институт (филиал) Военн...",NaN,"{""Полная занятость""}","{""Вооруженные силы РФ (РВСН)""}",a1,NaN,"{""Полный день""}","{Работоспособность,Обучаемость}",False,{Аналитик},80000.0


In [547]:
language_levels = {'a1': 1, 'a2': 2, 'b1': 3, 'b2': 4, 'c1': 5, 'c2': 6, 'l1': 7}
df['language_eng_encoded'] = df['language_eng'].map(language_levels)

In [549]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4024 entries, 10 to 9117
Data columns (total 20 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   title                     4024 non-null   object 
 1   age                       3760 non-null   float64
 2   gender                    4024 non-null   object 
 3   photo                     4024 non-null   bool   
 4   total_experience          3896 non-null   float64
 5   citizenship               4024 non-null   object 
 6   area                      4024 non-null   object 
 7   level_education           4024 non-null   object 
 8   university                4024 non-null   object 
 9   count_additional_courses  1748 non-null   float64
 10  employments               4024 non-null   object 
 11  experience                4024 non-null   object 
 12  language_eng              3062 non-null   object 
 13  language_zho              28 non-null     object 
 14  schedules   

In [550]:
df = df.drop(columns=[
    'language_zho', 'language_eng'
])
df = df.dropna(subset=['age', 'total_experience', 'language_eng_encoded'])

In [552]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2842 entries, 132 to 9117
Data columns (total 18 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   title                     2842 non-null   object 
 1   age                       2842 non-null   float64
 2   gender                    2842 non-null   object 
 3   photo                     2842 non-null   bool   
 4   total_experience          2842 non-null   float64
 5   citizenship               2842 non-null   object 
 6   area                      2842 non-null   object 
 7   level_education           2842 non-null   object 
 8   university                2842 non-null   object 
 9   count_additional_courses  1337 non-null   float64
 10  employments               2842 non-null   object 
 11  experience                2842 non-null   object 
 12  schedules                 2842 non-null   object 
 13  skill_set                 2842 non-null   object 
 14  is_driver  

In [555]:
df["has_photo"] = df["photo"].astype(int)
df["has_driver_license"] = df["is_driver"].astype(int)

In [556]:
df = df.drop(columns=[
    'title'
])

In [558]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2842 entries, 132 to 9117
Data columns (total 19 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   age                       2842 non-null   float64
 1   gender                    2842 non-null   object 
 2   photo                     2842 non-null   bool   
 3   total_experience          2842 non-null   float64
 4   citizenship               2842 non-null   object 
 5   area                      2842 non-null   object 
 6   level_education           2842 non-null   object 
 7   university                2842 non-null   object 
 8   count_additional_courses  1337 non-null   float64
 9   employments               2842 non-null   object 
 10  experience                2842 non-null   object 
 11  schedules                 2842 non-null   object 
 12  skill_set                 2842 non-null   object 
 13  is_driver                 2842 non-null   bool   
 14  professiona

In [561]:
df = df.drop(columns=[
    'is_driver', 'photo'
])
df = df.rename(columns={
    "has_driver_license": "is_driver",
    "has_photo": "photo", 
    'language_eng_encoded': 'language_eng'
})


In [563]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2842 entries, 132 to 9117
Data columns (total 17 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   age                       2842 non-null   float64
 1   gender                    2842 non-null   object 
 2   total_experience          2842 non-null   float64
 3   citizenship               2842 non-null   object 
 4   area                      2842 non-null   object 
 5   level_education           2842 non-null   object 
 6   university                2842 non-null   object 
 7   count_additional_courses  1337 non-null   float64
 8   employments               2842 non-null   object 
 9   experience                2842 non-null   object 
 10  schedules                 2842 non-null   object 
 11  skill_set                 2842 non-null   object 
 12  professional_roles        2842 non-null   object 
 13  russian_salary            2842 non-null   float64
 14  language_en

In [564]:
df = pd.get_dummies(df, columns=['gender'], dtype=int)
df.head()

,age,total_experience,citizenship,area,level_education,university,count_additional_courses,employments,experience,schedules,skill_set,professional_roles,russian_salary,language_eng,photo,is_driver,gender_female,gender_male
132,26.0,31.0,{Россия},Москва,Неоконченное высшее,"{""Московский государственный университет имени...",NaN,"{""Полная занятость"",""Частичная занятость"",""Про...",{Техноласточка},"{""Полный день"",""Сменный график"",""Гибкий график...","{""Аналитическое мышление"",Python,""Английский я...",{Аналитик},160000.0,5.0,1,0,0,1
164,51.0,355.0,{Россия},Москва,Высшее,"{""Серпуховской военный институт (филиал) Военн...",NaN,"{""Полная занятость""}","{""Вооруженные силы РФ (РВСН)""}","{""Полный день""}","{Работоспособность,Обучаемость}",{Аналитик},80000.0,1.0,0,0,0,1
219,40.0,224.0,{Россия},Иркутск,Высшее,"{""ГАПОУ Иркутской области «Центр обучения и со...",NaN,"{""Полная занятость""}","{""Газпромбанк (АО)"",""Банк ВТБ (ПАО)""}","{""Полный день""}",{},"{Аналитик,Маркетолог-аналитик,Экономист}",70000.0,1.0,0,1,1,0
273,31.0,33.0,{Россия},Москва,Высшее,"{""Московский государственный технический униве...",NaN,"{""Полная занятость"",""Частичная занятость""}","{""CORE (проект СбераБанка)"",""Сбербанк - Технол...","{""Полный день"",""Удаленная работа""}","{""Microsoft Office"",""Постановка задач разработ...","{Бизнес-аналитик,""Системный аналитик""}",180000.0,2.0,1,1,0,1
284,22.0,31.0,{Россия},Владивосток,Неоконченное высшее,"{""Дальневосточный федеральный университет"",Вла...",2.0,"{""Полная занятость""}","{""Сеть магазинов цифровой и бытовой техники DN...","{""Удаленная работа""}","{Docker,Python,SQL,""Английский язык"",""Математи...",{Дата-сайентист},200000.0,4.0,0,0,0,1


In [566]:
import re
from collections import Counter

def clean_city(city_str):
    city_str = city_str.strip('{}')
    city = re.findall(r'(?:[^,"]|"(?:\\.|[^"])*")+', city_str)
    city = [s.strip().strip('"') for s in city if s.strip()]
    return city

df['citizenship_cleaned'] = df['citizenship'].apply(clean_city)
df.head()

,age,total_experience,citizenship,area,level_education,university,count_additional_courses,employments,experience,schedules,skill_set,professional_roles,russian_salary,language_eng,photo,is_driver,gender_female,gender_male,citizenship_cleaned
132,26.0,31.0,{Россия},Москва,Неоконченное высшее,"{""Московский государственный университет имени...",NaN,"{""Полная занятость"",""Частичная занятость"",""Про...",{Техноласточка},"{""Полный день"",""Сменный график"",""Гибкий график...","{""Аналитическое мышление"",Python,""Английский я...",{Аналитик},160000.0,5.0,1,0,0,1,[Россия]
164,51.0,355.0,{Россия},Москва,Высшее,"{""Серпуховской военный институт (филиал) Военн...",NaN,"{""Полная занятость""}","{""Вооруженные силы РФ (РВСН)""}","{""Полный день""}","{Работоспособность,Обучаемость}",{Аналитик},80000.0,1.0,0,0,0,1,[Россия]
219,40.0,224.0,{Россия},Иркутск,Высшее,"{""ГАПОУ Иркутской области «Центр обучения и со...",NaN,"{""Полная занятость""}","{""Газпромбанк (АО)"",""Банк ВТБ (ПАО)""}","{""Полный день""}",{},"{Аналитик,Маркетолог-аналитик,Экономист}",70000.0,1.0,0,1,1,0,[Россия]
273,31.0,33.0,{Россия},Москва,Высшее,"{""Московский государственный технический униве...",NaN,"{""Полная занятость"",""Частичная занятость""}","{""CORE (проект СбераБанка)"",""Сбербанк - Технол...","{""Полный день"",""Удаленная работа""}","{""Microsoft Office"",""Постановка задач разработ...","{Бизнес-аналитик,""Системный аналитик""}",180000.0,2.0,1,1,0,1,[Россия]
284,22.0,31.0,{Россия},Владивосток,Неоконченное высшее,"{""Дальневосточный федеральный университет"",Вла...",2.0,"{""Полная занятость""}","{""Сеть магазинов цифровой и бытовой техники DN...","{""Удаленная работа""}","{Docker,Python,SQL,""Английский язык"",""Математи...",{Дата-сайентист},200000.0,4.0,0,0,0,1,[Россия]


In [567]:
df['citizenship_cleaned'].value_counts()

citizenship_cleaned
[Россия]                 2742
[Russia]                   34
[Казахстан]                15
[Беларусь]                 10
[Армения]                   7
[Казахстан, Россия]         4
[Другое, Россия]            4
[Украина]                   2
[Таджикистан]               2
[Израиль, Россия]           2
[Молдова, Россия]           2
[Кыргызстан]                2
[Узбекистан]                2
[Мексика]                   1
[Монголия]                  1
[Россия, Тунис]             1
[Нигерия]                   1
[Германия, Россия]          1
[Армения, Россия]           1
[Литва]                     1
[Россия, США]               1
[Кыргызстан, Россия]        1
[Молдова]                   1
[Беларусь, Россия]          1
[Россия, Таджикистан]       1
[Tajikistan]                1
[Другое]                    1
Name: count, dtype: int64

In [570]:
# Функция для проверки наличия России/Russia в списке
def is_russia(citizenships):
    if isinstance(citizenships, list):  # Если это список
        return any(
            isinstance(c, str) and 
            ('россия' in c.lower() or 'russia' in c.lower()) 
            for c in citizenships
        )
    return False  # Если не список (на случай NaN или других типов)

# Создаем столбцы
df['citizenship_russia'] = df['citizenship_cleaned'].apply(is_russia).astype(int)
df['citizenship_other'] = (~df['citizenship_cleaned'].apply(is_russia)).astype(int)

In [572]:
df = df.drop(columns=[
    'citizenship_cleaned', 'citizenship'
])
df['citizenship_russia'].value_counts()

citizenship_russia
1    2795
0      47
Name: count, dtype: int64

In [574]:
df.columns

Index(['age', 'total_experience', 'area', 'level_education', 'university',
       'count_additional_courses', 'employments', 'experience', 'schedules',
       'skill_set', 'professional_roles', 'russian_salary', 'language_eng',
       'photo', 'is_driver', 'gender_female', 'gender_male',
       'citizenship_russia', 'citizenship_other'],
      dtype='object')

In [575]:
df.head()

,age,total_experience,area,level_education,university,count_additional_courses,employments,experience,schedules,skill_set,professional_roles,russian_salary,language_eng,photo,is_driver,gender_female,gender_male,citizenship_russia,citizenship_other
132,26.0,31.0,Москва,Неоконченное высшее,"{""Московский государственный университет имени...",NaN,"{""Полная занятость"",""Частичная занятость"",""Про...",{Техноласточка},"{""Полный день"",""Сменный график"",""Гибкий график...","{""Аналитическое мышление"",Python,""Английский я...",{Аналитик},160000.0,5.0,1,0,0,1,1,0
164,51.0,355.0,Москва,Высшее,"{""Серпуховской военный институт (филиал) Военн...",NaN,"{""Полная занятость""}","{""Вооруженные силы РФ (РВСН)""}","{""Полный день""}","{Работоспособность,Обучаемость}",{Аналитик},80000.0,1.0,0,0,0,1,1,0
219,40.0,224.0,Иркутск,Высшее,"{""ГАПОУ Иркутской области «Центр обучения и со...",NaN,"{""Полная занятость""}","{""Газпромбанк (АО)"",""Банк ВТБ (ПАО)""}","{""Полный день""}",{},"{Аналитик,Маркетолог-аналитик,Экономист}",70000.0,1.0,0,1,1,0,1,0
273,31.0,33.0,Москва,Высшее,"{""Московский государственный технический униве...",NaN,"{""Полная занятость"",""Частичная занятость""}","{""CORE (проект СбераБанка)"",""Сбербанк - Технол...","{""Полный день"",""Удаленная работа""}","{""Microsoft Office"",""Постановка задач разработ...","{Бизнес-аналитик,""Системный аналитик""}",180000.0,2.0,1,1,0,1,1,0
284,22.0,31.0,Владивосток,Неоконченное высшее,"{""Дальневосточный федеральный университет"",Вла...",2.0,"{""Полная занятость""}","{""Сеть магазинов цифровой и бытовой техники DN...","{""Удаленная работа""}","{Docker,Python,SQL,""Английский язык"",""Математи...",{Дата-сайентист},200000.0,4.0,0,0,0,1,1,0


In [580]:
top_area = df['area'].value_counts().head(5).index
df['area_group'] = df['area'].apply(lambda x: x if x in top_area else 'other')
df = pd.get_dummies(df, columns=['area_group'], prefix='area', dtype=int)
df = df.drop(columns=['area'])

In [581]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2842 entries, 132 to 9117
Data columns (total 24 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   age                       2842 non-null   float64
 1   total_experience          2842 non-null   float64
 2   level_education           2842 non-null   object 
 3   university                2842 non-null   object 
 4   count_additional_courses  1337 non-null   float64
 5   employments               2842 non-null   object 
 6   experience                2842 non-null   object 
 7   schedules                 2842 non-null   object 
 8   skill_set                 2842 non-null   object 
 9   professional_roles        2842 non-null   object 
 10  russian_salary            2842 non-null   float64
 11  language_eng              2842 non-null   float64
 12  photo                     2842 non-null   int32  
 13  is_driver                 2842 non-null   int32  
 14  gender_fema

In [582]:
df = df.drop(columns=[
    'professional_roles'
])

In [583]:
def clean_skills(skill_str):
    skill_str = skill_str.strip('{}')
    skills = re.findall(r'(?:[^,"]|"(?:\\.|[^"])*")+', skill_str)
    skills = [s.strip().strip('"') for s in skills if s.strip()]
    return skills

df['skills_cleaned'] = df['skill_set'].apply(clean_skills)

selected_skills = [
    'MS Excel',
    'Работа с большим объемом информации',
    'Аналитическое мышление',
    'SQL',
    'MS PowerPoint',
    'Python',
    'BPMN',
    'Бизнес-анализ',
    'Разработка технических заданий',
    'Atlassian Jira',
    'Постановка задач разработчикам',
    'Системный анализ',
    'UML',
    'Power BI',
    'Atlassian Confluence',
    'Моделирование бизнес процессов',
    'Финансовый анализ',
    'MS SQL',
    'PostgreSQL',
    'Power Query',
    'MS Visio',
    'Статистический анализ',
    'Экономический анализ',
    'Управленческая отчетность',
    'Автоматизация процессов',
    'Визуализация данных',
    'Git',
    'Оптимизация бизнес-процессов',
    'Прогнозирование',
    'Анализ бизнес показателей',
    'Анализ финансовых показателей',
    '1С: Предприятие 8'
]


for skill in selected_skills:
    df[f'skill_{skill}'] = df['skills_cleaned'].apply(lambda x: 1 if skill in x else 0)

df = df.drop(columns=['skill_set', 'skills_cleaned'])

In [585]:
df.columns

Index(['age', 'total_experience', 'level_education', 'university',
       'count_additional_courses', 'employments', 'experience', 'schedules',
       'russian_salary', 'language_eng', 'photo', 'is_driver', 'gender_female',
       'gender_male', 'citizenship_russia', 'citizenship_other', 'area_other',
       'area_Казань', 'area_Краснодар', 'area_Москва', 'area_Нижний Новгород',
       'area_Санкт-Петербург', 'skill_MS Excel',
       'skill_Работа с большим объемом информации',
       'skill_Аналитическое мышление', 'skill_SQL', 'skill_MS PowerPoint',
       'skill_Python', 'skill_BPMN', 'skill_Бизнес-анализ',
       'skill_Разработка технических заданий', 'skill_Atlassian Jira',
       'skill_Постановка задач разработчикам', 'skill_Системный анализ',
       'skill_UML', 'skill_Power BI', 'skill_Atlassian Confluence',
       'skill_Моделирование бизнес процессов', 'skill_Финансовый анализ',
       'skill_MS SQL', 'skill_PostgreSQL', 'skill_Power Query',
       'skill_MS Visio', 'skill_

In [588]:
def clean_skills(skill_str):
    skill_str = skill_str.strip('{}')
    skills = re.findall(r'(?:[^,"]|"(?:\\.|[^"])*")+', skill_str)
    skills = [s.strip().strip('"') for s in skills if s.strip()]
    return skills

df['schedules_cleaned'] = df['schedules'].apply(clean_skills)

selected_schedules = [
    'Полный день',
    'Удаленная работа',
    'Гибкий график',
    'Сменный график',
    'Вахтовый метод'
]


for schedule in selected_schedules:
    df[f'schedules_{schedule}'] = df['schedules_cleaned'].apply(lambda x: 1 if schedule in x else 0)

df = df.drop(columns=['schedules', 'schedules_cleaned'])

In [589]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2842 entries, 132 to 9117
Data columns (total 58 columns):
 #   Column                                     Non-Null Count  Dtype  
---  ------                                     --------------  -----  
 0   age                                        2842 non-null   float64
 1   total_experience                           2842 non-null   float64
 2   level_education                            2842 non-null   object 
 3   university                                 2842 non-null   object 
 4   count_additional_courses                   1337 non-null   float64
 5   employments                                2842 non-null   object 
 6   experience                                 2842 non-null   object 
 7   russian_salary                             2842 non-null   float64
 8   language_eng                               2842 non-null   float64
 9   photo                                      2842 non-null   int32  
 10  is_driver                  

In [591]:
# Создаем mapping-словарь для преобразования
education_levels = {
    'Среднее': 1,
    'Среднее специальное': 2,
    'Неоконченное высшее': 3,
    'Incomplete higher': 3,
    'Бакалавр': 4,
    'Высшее': 4,
    'Bachelor': 4,
    'Higher': 4,  # Предполагаем, что это эквивалент бакалавра
    'Магистр': 5,
    'Master': 5,
    'Кандидат наук': 6,
    'PhD': 6,
    'Доктор наук': 7
}

# Применяем преобразование
df['education_level'] = df['level_education'].map(education_levels)

# Проверяем распределение
print(df['education_level'].value_counts().sort_index())

education_level
1      12
2      88
3     162
4    2105
5     429
6      42
7       4
Name: count, dtype: int64


In [592]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2842 entries, 132 to 9117
Data columns (total 59 columns):
 #   Column                                     Non-Null Count  Dtype  
---  ------                                     --------------  -----  
 0   age                                        2842 non-null   float64
 1   total_experience                           2842 non-null   float64
 2   level_education                            2842 non-null   object 
 3   university                                 2842 non-null   object 
 4   count_additional_courses                   1337 non-null   float64
 5   employments                                2842 non-null   object 
 6   experience                                 2842 non-null   object 
 7   russian_salary                             2842 non-null   float64
 8   language_eng                               2842 non-null   float64
 9   photo                                      2842 non-null   int32  
 10  is_driver                  

In [594]:
df = df.drop(columns=[
    'level_education'
])
df = df.rename(columns={
    "education_level": "level_education"
})

In [595]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2842 entries, 132 to 9117
Data columns (total 58 columns):
 #   Column                                     Non-Null Count  Dtype  
---  ------                                     --------------  -----  
 0   age                                        2842 non-null   float64
 1   total_experience                           2842 non-null   float64
 2   university                                 2842 non-null   object 
 3   count_additional_courses                   1337 non-null   float64
 4   employments                                2842 non-null   object 
 5   experience                                 2842 non-null   object 
 6   russian_salary                             2842 non-null   float64
 7   language_eng                               2842 non-null   float64
 8   photo                                      2842 non-null   int32  
 9   is_driver                                  2842 non-null   int32  
 10  gender_female              

In [598]:
def clean_skills(skill_str):
    skill_str = skill_str.strip('{}')
    skills = re.findall(r'(?:[^,"]|"(?:\\.|[^"])*")+', skill_str)
    skills = [s.strip().strip('"') for s in skills if s.strip()]
    return skills

df['experience_cleaned'] = df['experience'].apply(clean_skills)

selected_experience = [
    'Сбер',
    'Индивидуальное предпринимательство / частная практика / фриланс',
    'Розничная сеть',
    'Магнит',
    'Банк ВТБ',
    'Т-Банк',
    'Группа компаний',
    'Яндекс',
    'Ростелеком',
    'МТС',
    'Альфа-Банк',
    'Росбанк',
    'Россельхозбанк',
    'Ozon',
    'Билайн',
    'Интернет-магазин',
    'Тинькофф Банк',
    'Пепсико Россия',
    'РЖД',
    'НИУ ВШЭ',
    'Промсвязьбанк',
    'EY Russia',
    'Газпромбанк',
    'Raiffeisenbank',
    'X5 Retail Group',
    'Совкомбанк',
    'ФК Открытие',
    'ЦБ РФ',
    'МегаФон',
    'Банк Уралсиб',
    'Яндекс Практикум',
    'Почта России',
    'Philip Morris International',
    'VK',
    'Иннотех',
    'Банк Хоум Кредит',
    'Wildberries',
    'Аэропорт Домодедово',
    'Tele2',
    'Лента'
]


for experience in selected_experience:
    df[f'experience_{experience}'] = df['experience_cleaned'].apply(lambda x: 1 if experience in x else 0)

df = df.drop(columns=['experience', 'experience_cleaned'])

In [601]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2842 entries, 132 to 9117
Data columns (total 97 columns):
 #   Column                                                                      Non-Null Count  Dtype  
---  ------                                                                      --------------  -----  
 0   age                                                                         2842 non-null   float64
 1   total_experience                                                            2842 non-null   float64
 2   university                                                                  2842 non-null   object 
 3   count_additional_courses                                                    1337 non-null   float64
 4   employments                                                                 2842 non-null   object 
 5   russian_salary                                                              2842 non-null   float64
 6   language_eng                                       

In [603]:
def clean_skills(skill_str):
    skill_str = skill_str.strip('{}')
    skills = re.findall(r'(?:[^,"]|"(?:\\.|[^"])*")+', skill_str)
    skills = [s.strip().strip('"') for s in skills if s.strip()]
    return skills

df['employments_cleaned'] = df['employments'].apply(clean_skills)

selected_employments = [
    'Полная занятость',
    'Частичная занятость',
    'Проектная работа',
    'Стажировка'
]


for employments in selected_employments:
    df[f'employments_{employments}'] = df['employments_cleaned'].apply(lambda x: 1 if employments in x else 0)

df = df.drop(columns=['employments', 'employments_cleaned'])

In [605]:
def clean_skills(skill_str):
    skill_str = skill_str.strip('{}')
    skills = re.findall(r'(?:[^,"]|"(?:\\.|[^"])*")+', skill_str)
    skills = [s.strip().strip('"') for s in skills if s.strip()]
    return skills

df['university_cleaned'] = df['university'].apply(clean_skills)

selected_university = [
    "Финансовый университет при Правительстве Российской Федерации",
    "Национальный исследовательский университет 'Высшая школа экономики'",
    "Московский государственный университет им. М.В. Ломоносова",
    "Российский экономический университет им. Г.В. Плеханова",
    "Российская академия народного хозяйства и государственной службы при Президенте РФ",
    "Санкт-Петербургский государственный университет",
    "Московский авиационный институт (национальный исследовательский университет)",
    "Московский государственный технический университет им. Н.Э. Баумана",
    "Московский физико-технический институт (Государственный университет)",
    "Уральский федеральный университет имени первого Президента России Б.Н. Ельцина",
    "Национальный исследовательский ядерный университет 'МИФИ'",
    "Государственный университет управления",
    "Кубанский государственный университет",
    "Казанский (Приволжский) федеральный университет",
    "Санкт-Петербургский политехнический университет Петра Великого",
    "Воронежский государственный университет",
    "Нижегородский государственный университет им. Н.И. Лобачевского",
    "Российский государственный университет нефти и газа им. И.М. Губкина",
    "Московский энергетический институт (Национальный исследовательский университет)",
    "Российский университет дружбы народов"
]


for university in selected_university:
    df[f'university_{university}'] = df['university_cleaned'].apply(lambda x: 1 if university in x else 0)

df = df.drop(columns=['university', 'university_cleaned'])

In [606]:
df = df[(df['russian_salary'] >= 30000) & (df['russian_salary'] <= 500000)]

In [607]:
df1=df.copy()

In [609]:
df1 = df1.drop(columns=['count_additional_courses'])

In [610]:
df1.columns

Index(['age', 'total_experience', 'russian_salary', 'language_eng', 'photo',
       'is_driver', 'gender_female', 'gender_male', 'citizenship_russia',
       'citizenship_other',
       ...
       'university_Национальный исследовательский ядерный университет 'МИФИ'',
       'university_Государственный университет управления',
       'university_Кубанский государственный университет',
       'university_Казанский (Приволжский) федеральный университет',
       'university_Санкт-Петербургский политехнический университет Петра Великого',
       'university_Воронежский государственный университет',
       'university_Нижегородский государственный университет им. Н.И. Лобачевского',
       'university_Российский государственный университет нефти и газа им. И.М. Губкина',
       'university_Московский энергетический институт (Национальный исследовательский университет)',
       'university_Российский университет дружбы народов'],
      dtype='object', length=118)

In [612]:
# Референтые группы 
X = df1.drop(columns=['russian_salary', 'gender_male', 'citizenship_russia' ,
                     'area_Москва'])  # Предикторы
y = df1['russian_salary']  # Целевая переменная

X = sm.add_constant(X)

model1 = sm.OLS(y, X).fit()
print(model1.summary())

                            OLS Regression Results                            
Dep. Variable:         russian_salary   R-squared:                       0.407
Model:                            OLS   Adj. R-squared:                  0.385
Method:                 Least Squares   F-statistic:                     18.55
Date:                Tue, 08 Apr 2025   Prob (F-statistic):          1.80e-235
Time:                        02:39:27   Log-Likelihood:                -35348.
No. Observations:                2804   AIC:                         7.090e+04
Df Residuals:                    2703   BIC:                         7.150e+04
Df Model:                         100                                         
Covariance Type:            nonrobust                                         
                                                                                                    coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------

In [614]:
# Флаг для контроля остановки
all_significant = False

while not all_significant:
    model = sm.OLS(y, X).fit()
    p_values = model.pvalues.drop('const', errors='ignore')
    
    if (p_values <= 0.05).all():
        all_significant = True
    else:
        max_p_value = p_values.max()
        if max_p_value > 0.05:
            feature_to_drop = p_values.idxmax()
            print(f"Удаляем признак {feature_to_drop} (p-value = {max_p_value:.4f})")
            X = X.drop(columns=[feature_to_drop])
        else:
            all_significant = True

print(model.summary())

Удаляем признак skill_Экономический анализ (p-value = 0.9900)
Удаляем признак experience_Почта России (p-value = 0.9444)
Удаляем признак university_Московский авиационный институт (национальный исследовательский университет) (p-value = 0.9173)
Удаляем признак skill_Power BI (p-value = 0.8725)
Удаляем признак experience_ЦБ РФ (p-value = 0.9840)
Удаляем признак experience_Магнит (p-value = 0.9291)
Удаляем признак experience_Росбанк (p-value = 0.8655)
Удаляем признак university_Российский государственный университет нефти и газа им. И.М. Губкина (p-value = 0.8540)
Удаляем признак experience_Philip Morris International (p-value = 0.9543)
Удаляем признак university_Финансовый университет при Правительстве Российской Федерации (p-value = 0.8464)
Удаляем признак university_Национальный исследовательский университет 'Высшая школа экономики' (p-value = 0.8981)
Удаляем признак skill_Разработка технических заданий (p-value = 0.8410)
Удаляем признак experience_X5 Retail Group (p-value = 0.9335)
Уд

In [616]:
def calculate_vifs(X):
    vif_data = pd.DataFrame()
    # Исключаем 'const' из расчета VIF
    features = [col for col in X.columns if col != 'const']
    vif_data["feature"] = features
    vif_data["VIF"] = [variance_inflation_factor(X[features].values, i) for i in range(len(features))]
    return vif_data
    
high_vif = True
while high_vif:
    vifs = calculate_vifs(X)
    max_vif = vifs['VIF'].max()
    
    if max_vif > 5:  # Порог VIF
        feature_to_drop = vifs.loc[vifs['VIF'].idxmax(), 'feature']
        print(f"Удаляем признак {feature_to_drop} (VIF = {max_vif:.2f})")
        X = X.drop(columns=[feature_to_drop])
    else:
        high_vif = False

# Финальная модель
final_model = sm.OLS(y, X).fit()
print(final_model.summary())

# Вывод итоговых VIF
print("\nФинальные VIF значения:")
print(calculate_vifs(X).sort_values('VIF', ascending=False))

Удаляем признак age (VIF = 46.90)
Удаляем признак level_education (VIF = 13.28)
                            OLS Regression Results                            
Dep. Variable:         russian_salary   R-squared:                       0.382
Model:                            OLS   Adj. R-squared:                  0.372
Method:                 Least Squares   F-statistic:                     38.73
Date:                Tue, 08 Apr 2025   Prob (F-statistic):          2.38e-251
Time:                        02:39:31   Log-Likelihood:                -35406.
No. Observations:                2804   AIC:                         7.090e+04
Df Residuals:                    2759   BIC:                         7.117e+04
Df Model:                          44                                         
Covariance Type:            nonrobust                                         
                                                                                      coef    std err          t      P>|t|      [

In [620]:
X.corrwith(y).sort_values()

C:\Users\User\anaconda3\Lib\site-packages\numpy\lib\function_base.py:2897: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
C:\Users\User\anaconda3\Lib\site-packages\numpy\lib\function_base.py:2898: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]


area_other                                                                        -0.247417
gender_female                                                                     -0.200720
employments_Стажировка                                                            -0.169978
employments_Частичная занятость                                                   -0.159037
schedules_Сменный график                                                          -0.155103
skill_Работа с большим объемом информации                                         -0.106128
area_Санкт-Петербург                                                              -0.083962
area_Краснодар                                                                    -0.054880
area_Казань                                                                       -0.032601
area_Нижний Новгород                                                              -0.031036
experience_Индивидуальное предпринимательство / частная практика / фриланс      

In [622]:
df.columns

Index(['age', 'total_experience', 'count_additional_courses', 'russian_salary',
       'language_eng', 'photo', 'is_driver', 'gender_female', 'gender_male',
       'citizenship_russia',
       ...
       'university_Национальный исследовательский ядерный университет 'МИФИ'',
       'university_Государственный университет управления',
       'university_Кубанский государственный университет',
       'university_Казанский (Приволжский) федеральный университет',
       'university_Санкт-Петербургский политехнический университет Петра Великого',
       'university_Воронежский государственный университет',
       'university_Нижегородский государственный университет им. Н.И. Лобачевского',
       'university_Российский государственный университет нефти и газа им. И.М. Губкина',
       'university_Московский энергетический институт (Национальный исследовательский университет)',
       'university_Российский университет дружбы народов'],
      dtype='object', length=119)

In [624]:
df2=df.copy()
df2['count_additional_courses'].fillna(df2['count_additional_courses'].median(), inplace=True)

C:\Users\User\AppData\Local\Temp\ipykernel_18632\1888289706.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df2['count_additional_courses'].fillna(df2['count_additional_courses'].median(), inplace=True)


In [625]:
df2['count_additional_courses'].value_counts()

count_additional_courses
2.0     1726
1.0      482
3.0      181
4.0      113
6.0       69
5.0       65
7.0       38
8.0       26
10.0      22
11.0      17
9.0       14
12.0      11
14.0      11
19.0       6
15.0       5
21.0       4
13.0       4
17.0       4
52.0       2
18.0       2
16.0       1
22.0       1
Name: count, dtype: int64

In [626]:
# Референтые группы 
X = df2.drop(columns=['russian_salary', 'gender_male', 'citizenship_russia' ,
                     'area_Москва'])  # Предикторы
y = df2['russian_salary']  # Целевая переменная

X = sm.add_constant(X)

model1 = sm.OLS(y, X).fit()
print(model1.summary())

                            OLS Regression Results                            
Dep. Variable:         russian_salary   R-squared:                       0.408
Model:                            OLS   Adj. R-squared:                  0.386
Method:                 Least Squares   F-statistic:                     18.43
Date:                Tue, 08 Apr 2025   Prob (F-statistic):          1.00e-235
Time:                        02:39:32   Log-Likelihood:                -35345.
No. Observations:                2804   AIC:                         7.089e+04
Df Residuals:                    2702   BIC:                         7.150e+04
Df Model:                         101                                         
Covariance Type:            nonrobust                                         
                                                                                                    coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------

In [627]:
# Флаг для контроля остановки
all_significant = False

while not all_significant:
    model = sm.OLS(y, X).fit()
    p_values = model.pvalues.drop('const', errors='ignore')
    
    if (p_values <= 0.05).all():
        all_significant = True
    else:
        max_p_value = p_values.max()
        if max_p_value > 0.05:
            feature_to_drop = p_values.idxmax()
            print(f"Удаляем признак {feature_to_drop} (p-value = {max_p_value:.4f})")
            X = X.drop(columns=[feature_to_drop])
        else:
            all_significant = True

print(model.summary())

Удаляем признак university_Московский авиационный институт (национальный исследовательский университет) (p-value = 0.9493)
Удаляем признак experience_НИУ ВШЭ (p-value = 0.9495)
Удаляем признак skill_Экономический анализ (p-value = 0.9402)
Удаляем признак experience_Магнит (p-value = 0.9676)
Удаляем признак skill_Power BI (p-value = 0.9341)
Удаляем признак experience_Tele2 (p-value = 0.9570)
Удаляем признак experience_X5 Retail Group (p-value = 0.9542)
Удаляем признак experience_Почта России (p-value = 0.9102)
Удаляем признак experience_Росбанк (p-value = 0.8943)
Удаляем признак university_Финансовый университет при Правительстве Российской Федерации (p-value = 0.8881)
Удаляем признак skill_Анализ финансовых показателей (p-value = 0.8548)
Удаляем признак skill_Разработка технических заданий (p-value = 0.8476)
Удаляем признак experience_ФК Открытие (p-value = 0.8558)
Удаляем признак experience_Банк Уралсиб (p-value = 0.8606)
Удаляем признак university_Российский государственный университ

In [628]:
def calculate_vifs(X):
    vif_data = pd.DataFrame()
    # Исключаем 'const' из расчета VIF
    features = [col for col in X.columns if col != 'const']
    vif_data["feature"] = features
    vif_data["VIF"] = [variance_inflation_factor(X[features].values, i) for i in range(len(features))]
    return vif_data
    
high_vif = True
while high_vif:
    vifs = calculate_vifs(X)
    max_vif = vifs['VIF'].max()
    
    if max_vif > 10:  # Порог VIF
        feature_to_drop = vifs.loc[vifs['VIF'].idxmax(), 'feature']
        print(f"Удаляем признак {feature_to_drop} (VIF = {max_vif:.2f})")
        X = X.drop(columns=[feature_to_drop])
    else:
        high_vif = False

# Финальная модель
final_model = sm.OLS(y, X).fit()
print(final_model.summary())

# Вывод итоговых VIF
print("\nФинальные VIF значения:")
print(calculate_vifs(X).sort_values('VIF', ascending=False))

Удаляем признак age (VIF = 47.08)
Удаляем признак level_education (VIF = 13.48)
                            OLS Regression Results                            
Dep. Variable:         russian_salary   R-squared:                       0.386
Model:                            OLS   Adj. R-squared:                  0.375
Method:                 Least Squares   F-statistic:                     36.83
Date:                Tue, 08 Apr 2025   Prob (F-statistic):          9.55e-253
Time:                        02:39:37   Log-Likelihood:                -35397.
No. Observations:                2804   AIC:                         7.089e+04
Df Residuals:                    2756   BIC:                         7.117e+04
Df Model:                          47                                         
Covariance Type:            nonrobust                                         
                                                                                      coef    std err          t      P>|t|      [